In [ ]:
import time, os, sys
import matplotlib as mpl
import time
from PIL import Image
import skimage.io as io
import cv2
import scipy
import scipy.ndimage
from tifffile import imwrite
import tifffile as tif
import numpy as np
import skimage.io
from scipy import ndimage
from PIL import Image
import matplotlib.pyplot as plt
from tifffile import imwrite
import scipy
from scipy.ndimage import shift
from scipy.fft import fft

In [ ]:
file_path0 = 'REFstitch.tif'
contrast=np.zeros(60)
angle=np.zeros(60)

# Pre-processing

In [ ]:
def pre_processing(file_path, angle):
    LFM1=skimage.io.imread(file_path)
    img_rotated=ndimage.rotate(LFM1,angle,reshape=False)
    LFM1=img_rotated.astype(np.int32)
    h,w=LFM1.shape[:2]
    fraction1=15/14.12
    fraction2=15/14.12
    img_resized=np.array(Image.fromarray(LFM1,mode='I').resize((int(fraction1*w),int(fraction2*h)),Image.Resampling.BILINEAR))
    return img_resized

angle0=0
calibration_image=pre_processing(file_path0,angle0)

section=calibration_image
column_sums=np.sum(section,axis=0)
columns=np.arange(len(column_sums))

plt.figure(figsize=(10, 6))
plt.plot(columns, column_sums)
plt.title('Sum of Pixels per Column')
plt.xlabel('Column Index')
plt.ylabel('Sum of Pixels')
#plt.savefig("sum0.svg", format='svg')
plt.show()

#Rotation angle correction

In [ ]:
or ii in range(60):    
    LFM32=calibration_image.astype(np.int32)
    LFMcrop=LFM32
    angle[ii]=-1+ii/10
    LFMcrop=ndimage.rotate(LFMcrop,angle[ii],reshape=False)
    h,w=LFMcrop.shape[:2]
    fraction=15/14.12
    img_resized=np.array(Image.fromarray(LFMcrop,mode='I').resize((int(fraction*w),int(fraction*h)),Image.BILINEAR))
    image=img_resized   
    aa=(np.sum(image[750:1600,750:1600],0))
    contrast[ii]=(np.max(aa)-np.min(aa))/(np.max(aa)+np.min(aa))

plt.subplot(1,2,1)
plt.plot(aa[0:500])
plt.subplot(1,2,2)
plt.plot(angle,contrast)
MM=angle[np.argmax(contrast)]     #angle with best contrast 
print(MM)

file_path0='REFstitch.tif'
angle0=MM # Example angle
calibration_image=pre_processing(file_path0,angle0)

section=calibration_image
column_sums=np.sum(section, axis=0)
columns=np.arange(len(column_sums))

plt.figure(figsize=(10,6))
plt.plot(columns,column_sums)
plt.title('Sum of Pixels per Column')
plt.xlabel('Column Index')
plt.ylabel('Sum of Pixels')
plt.show()

# Fast Fourier transform to detect microlens pitch

In [ ]:
fft_result=fft(column_sums)
fft_abs=np.abs(fft_result)
freq=np.fft.fftfreq(len(column_sums))

plt.figure(figsize=(10, 6))
plt.plot(freq, fft_abs)
plt.title('Fast Fourier Transform')
plt.xlabel('Frequency')
plt.ylabel('Amplitude')
#plt.savefig("FFT0.svg", format='svg')
plt.show()

In [ ]:
peak_freq_indices=np.argpartition(fft_abs,-3)[-3:]
sorted_peak_indices=peak_freq_indices[np.argsort(fft_abs[peak_freq_indices])][::-1]
peak_freqs=freq[sorted_peak_indices]

peak2_freq=peak_freqs[1]
if peak2_freq!=0:
    peak2_period=1/peak2_freq
    print(f"La période du signal pour le deuxième pic est {peak2_period}")
else:
    print("Le deuxième pic a une fréquence de zéro, donc la période n'est pas définie.")

plt.figure(figsize=(10,6))
plt.rcParams.update({'font.size':12})
plt.plot(freq,fft_abs)
plt.plot(peak_freqs[1:],fft_abs[sorted_peak_indices][1:],'bo')  # Ajoute des points bleus
plt.title('FFT ')
plt.xlabel('Frequency')
plt.ylabel('Amplitude')
#plt.savefig("FFT0-point.svg", format='svg')
plt.show()